## predicting the health uninsured in the US

This project will call data from the American Community Survey Public Use MicroSample(PUMS) API. The most recent survey publicly availble at this API is from 2019. (Note data from 2020 are available to download in csv files).

1. American Community Survey (ACS) (census.gov).
https://www.census.gov/programs-surveys/acs/
2. American Community Survey Data via API (census.gov).
https://www.census.gov/programs-surveys/acs/data/data-via-api.html

The goal of this project is to predict whether an individual has health insurance (or not) based on demographic data in the PUMS. 

In [1]:
# import libraries
import os
import glob
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

There are hundreds of variables in the PUMS dataset. Load in the variable names (code) and their descriptions (label) from the API.

In [2]:
# load all variable names from census PUMS (public use microsample) API
load_dotenv()
API_KEY  = os.getenv('CENSUS_API_KEY')
host     = 'https://api.census.gov/data'
year     = 2019
dataset  = 'acs/acs1/pums/variables'
base_url = "/".join([host, str(year), dataset]) 
r        = requests.get(base_url)
rlists   = r.json()
code     = [item[0] for item in rlists if item[0].isupper()] #code in API
label    = [item[1].lower() for item in rlists if item[0].isupper()] # description for each code
nsamples = 500

codedict = {} # redudant with code, label but may make life easier later on
count    = 0
for lab in label:
    codedict[code[count]] = lab
    count += 1

When calling the Census API, a maximum of 50 variables can be requested at one time. The data are also easily accessed state-by-state. The PUMS represents ~1% of the population. If we want to use data from all states, we could start by subsampling from availbable records. Set the number of samples (nsamples) in getCensusDataByState().

One approach is to request data from a list of variables for a specific state in one call. The function callCensusAPI() below is meant to return a dataframe with variables in 'select_codes' from one state ('stateid'). A second function, getCensusDataByState() is meant to request data for all variables for one state in multiple calls and merge that data into one single dataframe for that state.

In [3]:
# function to get data from Census API, Public Access MicroData Samples using list of codes (max 50), state id number 
def callCensusApi(API_KEY,year,select_codes,stateid):
    host     = 'https://api.census.gov/data'
    dataset  = 'acs/acs1/pums'
    query    = '?get='
    variable = ','.join(select_codes)
    base_url = "/".join([host, str(year), dataset]) + query + variable + '&for=state:' + stateid + '&key=' + API_KEY
    callbeg  = time.time()
    r        = requests.get(base_url) 
    colnames = variable.split(',')
    colnames.append('state')
    df       = pd.DataFrame(columns=colnames, data=r.json()[1:]) 
    tElapsed = time.time()-callbeg
    return (tElapsed,df)

In [4]:
# get full dataframe (all variables in code) for one state. uses callCensusAPI function above
def getCensusDataByState(state_id):
    max_var_call = 50
    start_var_n  = 0
    end_var_n = start_var_n + max_var_call
    totalTime = 0
    while end_var_n <= len(code): 
        tElapsed, hdf = callCensusApi(API_KEY,year,code[start_var_n:end_var_n],state_id)
        if start_var_n == 0:
            df = hdf.sample(nsamples)
            rowid = df.index
            df = df.reset_index()
        else:
            sub = hdf.iloc[rowid]
            sub = sub.reset_index()
            df = pd.merge(df,sub,how='left',on='index')
            del sub
        totalTime = totalTime + tElapsed
        start_var_n = end_var_n + 1
        end_var_n = start_var_n + max_var_call
    if df.shape[1] < len(code):
        tElapsed, hdf = callCensusApi(API_KEY,year,code[start_var_n:len(code)],state_id)
        sub = hdf.iloc[rowid]
        sub = sub.reset_index()
        df  = pd.merge(df,sub,how='left',on='index')      
        totalTime = totalTime + tElapsed
        del hdf
    return (df,totalTime)

With the functions above, we can loop through our states of interest and save a dataframe for each if it does not exist locally yet.

In [5]:
datadir = os.path.join(os.getcwd(),"data")
try:
    os.mkdir(datadir)
except:
    print('already exists')

already exists


In [21]:
# call API for states in loop and save csv locally if it doesn't exist yet:
n_states = 51
for state in range(4,n_states+1):
    state_id  =  f"{state:02}"
    fname = os.path.join(datadir,'state' + state_id + '_subsample' + str(nsamples) + '.csv')
    if ~os.path.exists(fname):
        df,totalTime = getCensusDataByState(state_id)
        # save the data to a new csv file
        df.to_csv(fname)
        print('state' + state_id + ': ' + str(totalTime))
    del df
    del fname
    del state_id

C:\Users\kacie\AppData\Local\Temp/ipykernel_2200/2399290971.py:16: FutureWarning: Passing 'suffixes' which cause duplicate columns {'state_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df,sub,how='left',on='index')


state04: 129.3967263698578
state05: 57.973498582839966
state06: 705.9580404758453


JSONDecodeError: [Errno Expecting value] : 0

In [11]:
# load saved csv files:
filenames = glob.glob(os.path.join(datadir,'state' + '*' + str(nsamples) + '.csv'))
data_all = pd.concat((pd.read_csv(file) for file in filenames)).reset_index(drop = True) # Import

['C:\\Users\\kacie\\Documents\\ds\\census_healthinsurance\\data\\state01_subsample500.csv', 'C:\\Users\\kacie\\Documents\\ds\\census_healthinsurance\\data\\state02_subsample500.csv']


In [12]:
# check basic attributes of saved data:
print('shape of the subsampled dataframe: ' + str(data_all.shape[0]) + ' by ' + str(data_all.shape[1]))
print(data_all.head())

shape of the subsampled dataframe: 1000 by 518
   Unnamed: 0  index  HHLANP  FBATHP  DRIVESP  WGTP23  WGTP22  WGTP25  WGTP24  \
0           0  35478    1200       0        1     167     164     241     130   
1           1  41677    9500       0        1       8      19       6       8   
2           2  48485    9500       0        0     153      78      19     113   
3           3  25087    9500       0        0      71      61      21     124   
4           4  43537    9500       0        1      19      62      71      78   

   RACNH  ...  FDOUTP  PERNP  SCH  state_y.4  TEL  TEN  MLPI  MLPJ  MLPK  \
0      0  ...       0 -10001    2          1    1    2    -1    -1    -1   
1      0  ...       0      0    2          1    1    3    -1    -1    -1   
2      0  ...       0 -10001    2          1    1    1    -1    -1    -1   
3      0  ...       0 -10001    2          1    1    3    -1    -1    -1   
4      0  ...       0      0    2          1    1    1    -1    -1    -1   

   state 

In [13]:
# call api for more details on a specific variable
def variableDetails(codename):
    base_url = "/".join([host, str(year), dataset, codename + '.json']) 
    r        = requests.get(base_url)
    detail   = r.json()
    return detail

In [14]:
# check for extra columns that may have appeared from the dataframe merger:
for column in data_all:
    if column not in codedict.keys():
        if column.__contains__('.'):
            data_all.drop(column,axis=1,inplace=True)
        else:
            print('keeping ' + column)

print(data_all.shape)

keeping Unnamed: 0
keeping index
keeping state_x
keeping state_y
keeping state
(1000, 510)


In [15]:
# based on the way we merged state_x and state_y should be redundant
if data_all['state_x'].equals(data_all['state_y']):
    data_all.drop('state_y',inplace=True,axis=1)

In [16]:
# the state column should be incomplete based on the way we merged, drop if it has more missing values than 'state_x'
if data_all['state'].isna().sum() > data_all['state_x'].isna().sum():
    data_all.drop('state',axis=1,inplace=True)
# drop Unnamed: 0 because it's not useful at this point (from old indices)
data_all.drop('Unnamed: 0',axis=1,inplace=True)

In [17]:
# check for target variable (health insured or not):
for key, value in codedict.items():
    if value.__contains__('health insurance'):
        print(key + " : " + value)

PRIVCOV : private health insurance coverage recode
HICOV : health insurance coverage recode
FHICOVP : health insurance coverage recode allocation flag
FPRIVCOVP : private health insurance coverage recode allocation flag


In [18]:
# let's check details on health insurance coverage recode, and "...allocation flag"
codename = 'HICOV'
details = variableDetails(codename)
print('from ' + codename + ' : ')
for key,value in details['values']['item'].items():
    print(key, ':', value)

codename = 'FHICOVP'
print('from ' + codename + ' : ')
details = variableDetails(codename)
for key,value in details['values']['item'].items():
    print(key, ':', value)

from HICOV : 
2 : No health insurance coverage
1 : With health insurance coverage
from FHICOVP : 
0 : No
1 : Yes


In [19]:
# we can drop the HICOV column, since HICOV and FHICOVP have the same info
print(data_all.shape)
data_all.drop('FHICOVP',axis=1,inplace=True)
print(data_all.shape)

(1000, 508)
(1000, 507)


In [ ]:
missing = pd.concat([data_all.isnull().sum(), 100 * data_all.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count',ascending=False)

In [139]:
# check for missing values:
missing_values = []
for column in data_all:
    missing_values.append(data_all[column].isna().sum())
    if data_all[column].isna().sum() > 0:
        idx = code.index(column)
        print(code[idx] + ' : ' + label[idx])

OTHSVCEX : other internet service
GASP : gas cost (monthly cost, use adjhsg to adjust gasp to constant dollars)
WGTP11 : housing weight replicate 11
MAR : marital status
WGTP66 : housing weight replicate 66
FHIMRKSP : subsidized marketplace coverage allocation flag
HINS5 : tricare or other military health care
FMRGXP : first mortgage status allocation flag
FES : family type and employment status
WKL : when last worked
WGTP33 : housing weight replicate 33
WGTP43 : housing weight replicate 43
WGTP59 : housing weight replicate 59
PWGTP27 : person's weight replicate 27
PWGTP18 : person's weight replicate 18
AGS : sales of agriculture products (yearly sales, no adjustment factor is applied)
PWGTP62 : person's weight replicate 62
PWGTP52 : person's weight replicate 52
WGTP : housing unit weight
WGTP62 : housing weight replicate 62
WGTP80 : housing weight replicate 80
BDSP : number of bedrooms
FKITP : complete kitchen facilities allocation flag
FHINS5C : tricare coverage given through the eli